In [3]:
import pandas as pd
import re
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(palette="magma_r")

In [8]:
df_users = pd.read_csv("../..//data/processed/users.csv")
df_items = pd.read_csv("../..//data/processed/items.csv")


In [11]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 34 columns):
user_id                             120 non-null int64
style_age_range_group               120 non-null int64
style_size_preference_none          120 non-null int64
style_size_preference_petite        120 non-null int64
style_size_preference_extra_long    120 non-null int64
style_size_preference_plus          120 non-null int64
style_size_preference_maternity     120 non-null int64
style_size_preference_skipped       120 non-null int64
style_looks_wanted_skipped          120 non-null int64
style_most_important_active         120 non-null int64
style_most_important_any            120 non-null int64
style_most_important_beach          120 non-null int64
style_most_important_dress          120 non-null int64
style_most_important_bags           120 non-null int64
style_most_important_jeans          120 non-null int64
style_most_important_jump           120 non-null int64
style_most_impo

In [10]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9333 entries, 0 to 9332
Data columns (total 14 columns):
user_id                9333 non-null int64
brand_id               9333 non-null object
store_id               9333 non-null object
product_id             9333 non-null object
item_name_lower        9333 non-null object
product_category_id    9333 non-null int64
paid_price             9333 non-null float64
list_price             9333 non-null float64
size                   6085 non-null object
color_parsed           9333 non-null object
on_sale                9333 non-null bool
part_of_order          9333 non-null bool
top_brand              9333 non-null bool
color                  1030 non-null object
dtypes: bool(3), float64(2), int64(2), object(7)
memory usage: 829.5+ KB


In [ ]:
df_products = df_items[]